# Basic fitting for hyperfine beat (stage 1 bootstrap)

From prior work and data:

- Forbes, R. et al. (2018) ‘Quantum-beat photoelectron-imaging spectroscopy of Xe in the VUV’, Physical Review A, 97(6), p. 063417. Available at: https://doi.org/10.1103/PhysRevA.97.063417. arXiv: http://arxiv.org/abs/1803.01081, Authorea (original HTML version): https://doi.org/10.22541/au.156045380.07795038
- Data (OSF): https://osf.io/ds8mk/
- [Quantum Metrology with Photoelectrons (Github repo)](https://github.com/phockett/Quantum-Metrology-with-Photoelectrons), particularly the [Alignment 3 notebook](https://github.com/phockett/Quantum-Metrology-with-Photoelectrons/blob/master/Alignment/Alignment-3.ipynb). Functions from this notebook have been incorporated in the current project, under `qbanalysis.hyperfine`.

For basic fitting, try a stage 1 style bootstrap. In this case, set (arbitrary) parameters per final state for the probe, and fit these plus the hyperfine beat model parameters. This should allow for a match to a single set of hyperfine parameters for all observables.

- 14/06/24: basic fit for L=4/ROI-0 data working with Scipy. Next should add ionization model and use all states...

## Setup fitting model

Follow the modelling notebook, but wrap functions for fitting.

### Imports

In [1]:
# Load packages
# Main functions used herein from qbanalysis.hyperfine
from qbanalysis.hyperfine import *
import numpy as np
from epsproc.sphCalc import setBLMs

from pathlib import Path

dataPath = Path('/tmp/xe_analysis')
dataTypes = ['BLMall', 'BLMerr', 'BLMerrCycle']   # Read these types, should just do dir scan here.

# # Read from HDF5/NetCDF files
# # TO FIX: this should be identical to loadFinalDataset(dataPath), but gives slightly different plots - possibly complex/real/abs confusion?
# dataDict = {}
# for item in dataTypes:
#     dataDict[item] = IO.readXarray(fileName=f'Xe_dataset_{item}.nc', filePath=dataPath.as_posix()).real
#     dataDict[item].name = item

# Read from raw data files
from qbanalysis.dataset import loadFinalDataset
dataDict = loadFinalDataset(dataPath)

# Use Pandas and load Xe local data (ODS)
# These values were detemermined from the experimental data as detailed in ref. [4].
from qbanalysis.dataset import loadXeProps
xeProps = loadXeProps()

2024-06-14 10:02:07.850 | INFO     | qbanalysis.config:<module>:11 - PROJ_ROOT path is: /home/jovyan/code-share/github-share/Quantum-Beat_Photoelectron-Imaging_Spectroscopy_of_Xe_in_the_VUV
OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


* sparse not found, sparse matrix forms not available. 
* natsort not found, some sorting functions not available. 


* Setting plotter defaults with epsproc.basicPlotters.setPlotters(). Run directly to modify, or change options in local env.


* Set Holoviews with bokeh.
* pyevtk not found, VTK export not available. 
2024-06-14 10:02:13.483 | INFO     | qbanalysis.hyperfine:<module>:28 - Using uncertainties modules, Sympy maths functions will be forced to float outputs.
2024-06-14 10:02:13.552 | INFO     | qbanalysis.dataset:loadDataset:244 - Loaded data cpBasex_results_cycleSummed_rot90_quad1_ROI_results_with_FT_NFFT1024_hanningWindow_270717.mat.
2024-06-14 10:02:13.598 | INFO     | qbanalysis.dataset:loadDataset:244 - Loaded data cpBasex_results_allCycles_ROIs_with_FTs_NFFT1024_hanningWindow_270717.mat.
2024-06-14 10:02:13.912 | INFO     | qbanalysis.dataset:loadFinalDataset:220 - Processed data to Xarray OK.
2024-06-14 10:02:13.954 | INFO     | qbanalysis.dataset:loadXeProps:71 - Loaded Xe data from /home/jovyan/code-share/github-share/Quantum-Beat_Photoelectron-Imaging_Spectroscopy_of_Xe_in_the_VUV/dataLocal/Xe_data_table_fixedFractions.ods.

**Xe measured level splittings and the hyperfine constants.**
Statistical uncer

A/MHz      B/MHz   Splitting/cm−1
Isotope I   F   F′                                        
129     0.5 0.5 1.5  -5723+/-9  nan+/-nan  0.2863+/-0.0005
131     1.5 1.5 0.5  1697+/-30     -8+/-7  0.0855+/-0.0010
            2.5 1.5  1697+/-30     -8+/-7  0.1411+/-0.0029
                0.5  1697+/-30     -8+/-7  0.2276+/-0.0029

### Init parameters

Here use `xeProps` to set and define fit paramters. Note in the original work the splittings were determined by FT of the data, and A, B parameters via Eqn. 2 therein.

TODO: may want to use lmfit here for more flexibility.

In [2]:
# Set splittings
fitParamsCol = 'Splitting/cm−1'
xePropsFit = xeProps.copy()

xeSplittings = xePropsFit[fitParamsCol].to_numpy()

In [3]:
# Test beat model with changed params...
xeSplittings = np.random.randn(4)
xeSplittings

array([-0.18737903,  0.46271988, -0.40770107,  0.20519311])

In [4]:
xePropsFit[fitParamsCol] = 0.1*np.abs(xeSplittings)
xePropsFit

A/MHz      B/MHz  Splitting/cm−1
Isotope I   F   F′                                       
129     0.5 0.5 1.5  -5723+/-9  nan+/-nan        0.018738
131     1.5 1.5 0.5  1697+/-30     -8+/-7        0.046272
            2.5 1.5  1697+/-30     -8+/-7        0.040770
                0.5  1697+/-30     -8+/-7        0.020519

In [5]:
modelDict = computeModel(xePropsFit)
modelSum = computeModelSum(modelDict)['sum'] 
plotOpts = {'width':800}
(plotHyperfineModel(modelDict['129Xe'], **plotOpts) * plotHyperfineModel(modelDict['131Xe'], **plotOpts) * plotHyperfineModel(modelSum, **plotOpts)).opts(title="Isotope comparison + sum")

:Overlay
   .NdOverlay.I   :NdOverlay   [K,Q]
      :Spread   [t]   (129Xe,129Xe_std)
   .NdOverlay.II  :NdOverlay   [K,Q]
      :Curve   [t]   (129Xe,129Xe_std)
   .NdOverlay.III :NdOverlay   [K,Q]
      :Spread   [t]   (131Xe,131Xe_std)
   .NdOverlay.IV  :NdOverlay   [K,Q]
      :Curve   [t]   (131Xe,131Xe_std)
   .NdOverlay.V   :NdOverlay   [K,Q]
      :Spread   [t]   (sum,sum_std)
   .NdOverlay.VI  :NdOverlay   [K,Q]
      :Curve   [t]   (sum,sum_std)

In [6]:
# Test with ROI 0, l=4 case first...
dataDict['BLMall'].sel({'ROI':0,'l':4}).squeeze().hvplot()  #.line(x='t')

:Curve   [t]   (BLM input 0)

In [268]:
def residual(model,dataIn):
    """
    Calc least squares residual
    """
    res = (model - dataIn)**2  # Returning single value XR only in testing? Issue with dims?
                            # Ah, OK after fixing t-units
    # res = model.values - dataIn.values  # Force to NP, assumes matching size.

    return res

def setParams(xePropsIn, newVals, fitParamsCol = 'Splitting/cm−1'):
    """
    Replace single column in input dataframe with newVals.
    
    Note: no size checks here.
    """
    
    # Set splittings
    # fitParamsCol = 'Splitting/cm−1'
    xePropsUpdated = xePropsIn.copy()
    xePropsUpdated[fitParamsCol] = newVals
    
    return xePropsUpdated

# NOTE - setting trange here may be required.
# Fitting to full window tends to smooth out oscillations, may need to be more careful with residual func?
# trange=[0,200]  OK first part only
# trange=[0,500]  GOOD!
# trange=[0,800]  GOOD! Lower overall intensity than [0,500] case.
# trange=[0,1000]  GOOD! Lower overall intensity than [0,500] case.
# trange=None  OK, but t<0 data messes things up a bit.
def calcFitModel(xData, xePropsFit = None, dataDict = None, fitFlag=True, trange=[0,1000]):
    """
    Calc model and residual for Scipy fitting.
    
    Set fitFlag=False to return all model results.
    """
    
    # Update fit params
    xePropsFit = setParams(xePropsFit,xData)
    
    # Compute model
    modelDict = computeModel(xePropsFit, tIn=dataDict['BLMall'].t*1e-12)  # Note t-units in s!
    modelSum = computeModelSum(modelDict)['sum'] 

    # Compute residual
    dataIn = dataDict['BLMall'].sel({'ROI':0,'l':4}).copy()
    modelIn = modelSum.sel({'K':2}).squeeze(drop=True)
    # modelIn.values = unumpy.nominal_values(modelIn)  # Use nominal values only?
    # modelIn['t'].values = modelIn['t'].values.astype(int) 
    modelIn = modelIn.assign_coords({'t':modelIn['t'].values.astype(int)})  # Force to int to match input data

    # Optionally set trange
    if trange is not None:
        modelIn = modelIn.sel(t=slice(trange[0],trange[1]))
        dataIn = dataIn.sel(t=slice(trange[0],trange[1]))
    
    res = residual(modelIn, dataIn.squeeze())
    
    if fitFlag:
        return unumpy.nominal_values(res.values)
    else:
        return xePropsFit, modelDict, modelSum, modelIn, dataIn, res

# Test fit...

# Fit to residuals, just set to zero (NOT required for least_squares)
yData = xr.zeros_like(dataDict['BLMall'].sel({'ROI':0,'l':4}))
yData = yData.values

import scipy
# x0 = np.abs(np.random.random(4))  # Randomise inputs

xePropsFit = xeProps.copy()
x0 = cxePropsFit[fitParamsCol].to_numpy())  # Test with previous vals

fitOut = scipy.optimize.least_squares(calcFitModel, x0, bounds = (0.01,0.5),
                                      kwargs = {'xePropsFit':xePropsFit, 'dataDict':dataDict})
fitOut.success

True

In [269]:
dataIn

<xarray.DataArray 'BLM input 0' (m: 1, t: 97)>
array([[-0.05167322, -0.06306544, -0.04565688, -0.00785074, -0.1158776 ,
         0.00895966,  0.02378428, -0.27124133, -0.24741627, -0.19193347,
        -0.13308442, -0.07149924, -0.01906278, -0.00862747, -0.01267958,
        -0.03363959, -0.07707265, -0.12237209, -0.14167461, -0.15434793,
        -0.13277826, -0.09376219, -0.06309039, -0.03003538, -0.02132168,
        -0.0242818 , -0.05342943, -0.09056367, -0.13710848, -0.18361675,
        -0.17822407, -0.16950458, -0.14353127, -0.10566887, -0.06236894,
        -0.03275229, -0.0143162 , -0.03682004, -0.05230001, -0.06697774,
        -0.1021198 , -0.09943229, -0.08206377, -0.06411213, -0.0356055 ,
        -0.00212246,  0.00782249, -0.00708471, -0.03714555, -0.09606911,
        -0.15331731, -0.20250601, -0.23415317, -0.24289232, -0.20595094,
        -0.13173105, -0.06770041, -0.00723014,  0.02445007,  0.01700258,
         0.00995873, -0.0300354 , -0.07826857, -0.11163168, -0.15274686,
        -0.14884376, -0.11522001, -0.07316691, -0.02305059,  0.0056164 ,
         0.02128649,  0.01730564, -0.01354608, -0.0729367 , -0.14133225,
        -0.18903036, -0.20997328, -0.19366352, -0.17822534, -0.11404528,
        -0.06288428, -0.02578404, -0.01004083, -0.01508401, -0.0192957 ,
        -0.04289083, -0.06434454, -0.07928416, -0.08795014, -0.07110603,
        -0.04766487, -0.0281031 , -0.0195196 , -0.02318544, -0.03580319,
        -0.07720557, -0.13054183]])
Coordinates:
    ROI      int64 0
  * m        (m) int64 0
  * t        (t) int64 -70 -60 -50 -40 -30 -20 -10 ... 840 850 860 870 880 890
Attributes:
    dataType:   BLM
    long_name:  Beta parameters
    units:      arb
    harmonics:  {'dtype': 'sph', 'kind': 'complex', 'normType': 'ortho', 'csP...
    normType:   sph

In [270]:
modelPlot = splitUncertaintiesToDataset(modelIn)
# unumpy.nominal_values()
modelPlot.hvplot()

:NdOverlay   [Variable]
   :Curve   [t]   (value)

In [271]:
# modelFit['129Xe']

In [272]:
xeProps

A/MHz      B/MHz   Splitting/cm−1
Isotope I   F   F′                                        
129     0.5 0.5 1.5  -5723+/-9  nan+/-nan  0.2863+/-0.0005
131     1.5 1.5 0.5  1697+/-30     -8+/-7  0.0855+/-0.0010
            2.5 1.5  1697+/-30     -8+/-7  0.1411+/-0.0029
                0.5  1697+/-30     -8+/-7  0.2276+/-0.0029

In [273]:
x0

array([0.2863, 0.0855, 0.1411, 0.2276])

In [274]:
fitOut.x

array([0.29126303, 0.08466675, 0.1411    , 0.22949392])

In [275]:
fitOut

     message: `ftol` termination condition is satisfied.
     success: True
      status: 2
         fun: [ 8.541e-07  2.215e-05 ...  3.658e-04  5.021e-04]
           x: [ 2.913e-01  8.467e-02  1.411e-01  2.295e-01]
        cost: 2.3725378584475683e-05
         jac: [[ 0.000e+00  0.000e+00  0.000e+00  0.000e+00]
               [-9.299e-04  2.320e-04  0.000e+00 -5.137e-04]
               ...
               [ 5.836e-01 -3.171e-01  0.000e+00  2.142e-01]
               [ 7.487e-01 -3.187e-01  0.000e+00  9.719e-02]]
        grad: [-4.296e-09 -5.315e-08  0.000e+00  4.667e-09]
  optimality: 2.2076459731634488e-08
 active_mask: [0 0 0 0]
        nfev: 23
        njev: 12

In [276]:
# modelFitSum.sel(t=slice(0,600))

In [277]:
# modelFit['131Xe']

In [278]:
# Check results
xePropsFit, modelFit, modelFitSum, modelIn, dataIn, res = calcFitModel(fitOut.x, xePropsFit, dataDict, fitFlag=False)

# Model & components
(plotHyperfineModel(modelFit['129Xe'], **plotOpts) * plotHyperfineModel(modelFit['131Xe'], **plotOpts) * plotHyperfineModel(modelFitSum, **plotOpts)).opts(title="Isotope comparison + sum")

:Overlay
   .NdOverlay.I   :NdOverlay   [K,Q]
      :Spread   [t]   (129Xe,129Xe_std)
   .NdOverlay.II  :NdOverlay   [K,Q]
      :Curve   [t]   (129Xe,129Xe_std)
   .NdOverlay.III :NdOverlay   [K,Q]
      :Spread   [t]   (131Xe,131Xe_std)
   .NdOverlay.IV  :NdOverlay   [K,Q]
      :Curve   [t]   (131Xe,131Xe_std)
   .NdOverlay.V   :NdOverlay   [K,Q]
      :Spread   [t]   (sum,sum_std)
   .NdOverlay.VI  :NdOverlay   [K,Q]
      :Curve   [t]   (sum,sum_std)

In [279]:
# plotHyperfineModel(modelFit['129Xe'], **plotOpts) * plotHyperfineModel(modelFit['131Xe'], **plotOpts) * modelPlot.hvplot()

In [280]:
# modelFitSum

In [281]:
# dataIn.squeeze().hvplot() * modelSum.sel({'K':2}).squeeze(drop=True).hvplot.line(x='t')
# dataIn.squeeze().hvplot() * modelIn.hvplot()  # NEED TO SPLIT ON UNCERTAINTIES!!!

In [282]:
xeProps

A/MHz      B/MHz   Splitting/cm−1
Isotope I   F   F′                                        
129     0.5 0.5 1.5  -5723+/-9  nan+/-nan  0.2863+/-0.0005
131     1.5 1.5 0.5  1697+/-30     -8+/-7  0.0855+/-0.0010
            2.5 1.5  1697+/-30     -8+/-7  0.1411+/-0.0029
                0.5  1697+/-30     -8+/-7  0.2276+/-0.0029

In [283]:
xePropsFit

A/MHz      B/MHz  Splitting/cm−1
Isotope I   F   F′                                       
129     0.5 0.5 1.5  -5723+/-9  nan+/-nan        0.291263
131     1.5 1.5 0.5  1697+/-30     -8+/-7        0.084667
            2.5 1.5  1697+/-30     -8+/-7        0.141100
                0.5  1697+/-30     -8+/-7        0.229494

In [319]:
# res
import pandas as pd

fitParamsCol = 'Splitting/cm−1'
diffData = pd.DataFrame([xeProps[fitParamsCol], xePropsFit[fitParamsCol], xeProps[fitParamsCol]-xePropsFit[fitParamsCol]]).T
# diffData.columns.rename({n:item for n,item in enumerate(['original','fit','diff'])})
diffData.columns = ['original','fit','diff']
diffData['diff']= unumpy.nominal_values(diffData['diff'].values)
diffData

original       fit      diff
Isotope I   F   F′                                      
129     0.5 0.5 1.5  0.2863+/-0.0005  0.291263 -0.004963
131     1.5 1.5 0.5  0.0855+/-0.0010  0.084667  0.000833
            2.5 1.5  0.1411+/-0.0029    0.1411  0.000000
                0.5  0.2276+/-0.0029  0.229494 -0.001894

In [310]:
xeProps[fitParamsCol].rename({fitParamsCol:'fit'})

Isotope  I    F    F′ 
129      0.5  0.5  1.5    0.2863+/-0.0005
131      1.5  1.5  0.5    0.0855+/-0.0010
              2.5  1.5    0.1411+/-0.0029
                   0.5    0.2276+/-0.0029
Name: Splitting/cm−1, dtype: object

In [315]:
diffData.columns = ['original','fit','diff']

In [320]:
diffData

original       fit      diff
Isotope I   F   F′                                      
129     0.5 0.5 1.5  0.2863+/-0.0005  0.291263 -0.004963
131     1.5 1.5 0.5  0.0855+/-0.0010  0.084667  0.000833
            2.5 1.5  0.1411+/-0.0029    0.1411  0.000000
                0.5  0.2276+/-0.0029  0.229494 -0.001894

In [304]:
{n:item for n,item in enumerate(['original','fit','diff'])}

{0: 'original', 1: 'fit', 2: 'diff'}

In [291]:
from qbanalysis.plots import plotFinalDatasetBLMt
# plotFinalDatasetBLMt(**dataDict, **plotOpts) * plotHyperfineModel(modelFitSum, **plotOpts).select(K=2).opts(**plotOpts)
plotHyperfineModel(modelFitSum, **plotOpts).select(K=2).opts(**plotOpts) * plotFinalDatasetBLMt(**dataDict, **plotOpts)

:NdLayout   [l]
   :Overlay
      .NdOverlay.I   :NdOverlay   [K,Q]
         :Spread   [t]   (sum,sum_std)
      .NdOverlay.II  :NdOverlay   [K,Q]
         :Curve   [t]   (sum,sum_std)
      .NdOverlay.III :NdOverlay   [ROI]
         :Spread   [t]   (BLM per cycle,BLM per cycle_std)
      .NdOverlay.IV  :NdOverlay   [ROI]
         :Curve   [t]   (BLM input 0)

In [32]:
# Set fitting params
xePropsNP[:,2]

array([0.2863+/-0.0005, 0.0855+/-0.001, 0.1411+/-0.0029, 0.2276+/-0.0029],
      dtype=object)

In [33]:
xePropsNP[1,2] = 10

In [34]:
xeTest = xeProps.copy()

In [35]:
xeTest.xs(xePropsInd[1])['Splitting/cm−1']

0.0855+/-0.001

In [37]:
xeTest.xs(xePropsInd[1])['Splitting/cm−1'] = 10

In [38]:
xeTest

A/MHz      B/MHz   Splitting/cm−1
Isotope I   F   F′                                        
129     0.5 0.5 1.5  -5723+/-9  nan+/-nan  0.2863+/-0.0005
131     1.5 1.5 0.5  1697+/-30     -8+/-7               10
            2.5 1.5  1697+/-30     -8+/-7  0.1411+/-0.0029
                0.5  1697+/-30     -8+/-7  0.2276+/-0.0029

## SCRATCH

In [7]:
xeProps['A/MHz']

Isotope  I    F    F′ 
129      0.5  0.5  1.5    -5723+/-9
131      1.5  1.5  0.5    1697+/-30
              2.5  1.5    1697+/-30
                   0.5    1697+/-30
Name: A/MHz, dtype: object

In [11]:
xeProps.xs((131,1.5,1.5,0.5))

A/MHz                   1697+/-30
B/MHz                      -8+/-7
Splitting/cm−1    0.0855+/-0.0010
Name: (131, 1.5, 1.5, 0.5), dtype: object

In [12]:
xeProps.to_xarray()

<xarray.Dataset>
Dimensions:         (Isotope: 2, I: 2, F: 3, F′: 2)
Coordinates:
  * Isotope         (Isotope) int64 129 131
  * I               (I) float64 0.5 1.5
  * F               (F) float64 0.5 1.5 2.5
  * F′              (F′) float64 0.5 1.5
Data variables:
    A/MHz           (Isotope, I, F, F′) object nan -5723+/-9 ... 1697+/-30
    B/MHz           (Isotope, I, F, F′) object nan nan+/-nan ... -8+/-7 -8+/-7
    Splitting/cm−1  (Isotope, I, F, F′) object nan ... 0.1411+/-0.0029